## <a name="C4"> Projet 4-1 : Anticipez les besoins en consommation de bâtiments</a>

La transition vers des villes respectueuses de l'environnement est une priorité mondiale. Dans cette optique, la ville de Seattle s'est fixé un objectif ambitieux : devenir neutre en émissions de carbone d'ici 2050. Pour atteindre cet objectif, il est crucial de comprendre et de réduire la consommation d'énergie ainsi que les émissions de CO2 des bâtiments non destinés à l'habitation. Cependant, la collecte de données précises sur ces aspects peut s'avérer coûteuse. Dans ce contexte, notre mission consiste à anticiper les besoins en consommation de ces bâtiments en se basant sur des données structurelles disponibles, tout en évaluant l'intérêt de l'"ENERGY STAR Score" pour prédire les émissions de CO2. Cette tâche nécessitera une approche rigoureuse de modélisation et d'évaluation des performances.

## <a name="C4"> Mise en Place de l'Environnement de Travail</a>

Nous commençons par configurer notre environnement de travail en important les librairies nécessaires.

- Installation des Librairies

In [ ]:
# Importation des librairies nécessaires pour le projet

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns  
from scipy import stats  
import missingno as msno
import os
os.environ['OMP_NUM_THREADS'] = '1'
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import normaltest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.stats import boxcox
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import t
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score

# Afficher les graphiques directement dans le notebook
%matplotlib inline

# Configuration de Seaborn pour améliorer l'esthétique des graphiques
sns.set(style="whitegrid")

# Afficher plus de lignes et de colonnes
pd.set_option('display.max_row',1000)
pd.set_option('display.max_column',300)

# Indication d'importations réussies
print("Librairies importées avec succès !")

- Lecture des fichiers de données

Nous lisons ensuite les fichiers de données issus du site de seattle.gov.

In [ ]:
# Chargement des fichiers CSV dans des DataFrames pandas
data = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
# Indication d'importations réussies
print("Fichier importé avec succès !")

- Ecriture des fonctions utiles

Avant de préparer nos données, nous devons obtenir une description exacte de ces dernières. Pour cela, nous créons une fonction qui retourne un tableau comprenant toutes les informations dont nous avons besoin :

In [ ]:
def data_info(df: pd.DataFrame):
    desc = df.columns.to_frame(name="colonne").set_index('colonne')
    desc['nombre de valeurs non nulles'] = df.notnull().sum() 
    desc['nombre de valeurs uniques'] = df.nunique() 
    desc['type de donnée'] = df.dtypes 
    desc['nombre de valeurs nulles']=df.isna().sum()
    desc['pourcentage de valeurs nulles']=round((df.isna().sum()/df.shape[0]*100),2)
    return desc

# Indication de réussite
print("Fonction data_info créée avec succès !")

Nous souhaitons visualiser le taux de remplissage de certains indicateurs, pour cela nous créons une fonction qui affiche sur un histogramme le taux de remplissage des colonnes, avec la possibilité d'ignorer les colonnes non pertinentes.

In [ ]:
def fill_rates_plot(dataframe, ignore_columns=None):
    # Filtrer les colonnes si nécessaire
    if ignore_columns is not None:
        dataframe = dataframe.drop(columns=ignore_columns, errors='ignore')

    # Calcul du taux de remplissage pour chaque colonne
    fill_rates = dataframe.notnull().mean() * 100

    # Création de l'histogramme
    plt.figure(figsize=(25, 4))
    fill_rates.plot(kind='bar', color='steelblue')
    plt.title('Taux de remplissage des colonnes')
    plt.xlabel('Colonnes')
    plt.ylabel('Taux de remplissage (%)')
    plt.xticks(rotation=90)
    plt.grid(True, linestyle='--', linewidth=0.5, color='lightsteelblue')
    
    # Affichage de l'histogramme
    plt.show()
    
# Indication de réussite
print("Fonction fill_rates_plot créée avec succès !")

Nous souhaitons automatiser la suppression de colonnes dont le taux de remplissage est inférieur à un certain seuil.

In [ ]:
# Fonction de suppression de colonnes
def drop_columns(df, seuil=0.8):
    
    nb_observations = len(df)

    for colonne in df.columns:
        # Calculer le nombre de valeurs non nulles dans la colonne
        nb_non_nulles = df[colonne].count()

        # Calculer le taux de remplissage de la colonne
        taux_remplissage = nb_non_nulles / nb_observations

        # Si le taux de remplissage est inférieur au seuil, supprimer la colonne
        if taux_remplissage < seuil:
            del df[colonne]

    return df.head(5)

# Indication de réussite
print("Fonction drop_columns créée avec succès !")

Nous créons une fonciton qui écarte les individus dont la valeur des variables séléctionnées est inférieure à 0 et/ou supérieure à 100.

In [ ]:
def outliers_cleaning(df, colonnes, inf=0, sup=100):
    lignes_supprimees = 0 
    for colonne in colonnes:
        nb_lignes_avant = len(df)
        df = df[(df[colonne] >= inf) & (df[colonne] <= sup)]
        lignes_supprimees += nb_lignes_avant - len(df)
    
    return df, lignes_supprimees

# Indication de réussite
print("Fonction outliers_cleaning créée avec succès !")

Nous créons une fonction qui remplace les individus dont la valeur des variables séléctionnées est inférieure à 0 et/ou supérieure à 100 par NaN.

In [ ]:
def outliers_replacing(df, colonnes, inf=0, sup=100):
    valeurs_aberrantes_remplacees = 0 
    for colonne in colonnes:
        valeurs_aberrantes = (df[colonne] < inf) | (df[colonne] > sup)
        df.loc[valeurs_aberrantes, colonne] = np.nan
        valeurs_aberrantes_remplacees += valeurs_aberrantes.sum()
    
    return df, valeurs_aberrantes_remplacees

# Indication de réussite
print("Fonction outliers_replacing créée avec succès !")

Nous créons une fonction afin de visualiser la distribution des variables.

In [ ]:
def distribution_vizualisation(df, variables):

    for variable in variables:
        plt.figure(figsize=(12, 6))

        # Afficher l'histogramme et le graphique de densité
        plt.subplot(1, 2, 1)
        sns.histplot(df[variable], kde=True, color='blue', stat='density')
        plt.title('Distribution de ' + variable)
        plt.xlabel(variable)
        plt.ylabel('Densité')

        plt.tight_layout()
        plt.show()
        
# Indication de réussite
print("Fonction distribution_vizualisation2 créée avec succès !")

## <a name="C4"> Préparation des données </a>


Dans cette première partie du notebook, nous nous concentrerons sur l'observation initiale des données d'Open Food Facts. Notre objectif sera de comprendre la structure générale du jeu de données, d'identifier les types de variables présentes et de repérer les premiers signes de données manquantes ou aberrantes. Cette exploration préliminaire est cruciale pour déterminer les besoins spécifiques en nettoyage et en traitement des données, et poser les bases d'une analyse plus approfondie.

#### <font color='skyblue'> Observation des données </font>


Pour commencer nous visualisons les premieres lignes du datafreames pour avoir une idéee de sa structure.

In [ ]:
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))
data.head(3)

Ensuite nous utilisons notre fonction pour avoir plus d'infos sur le dataframes.

In [ ]:
data_info(data)

Nous observons le taux de remplissage.

In [ ]:
fill_rates_plot(data)

Nous observons les types de nos variables.

In [ ]:
# affichage des types object
object_types = data.select_dtypes(include=['object']).columns.tolist()
print("Il y a ",len(object_types),"variables catégorielles")
object_types

In [ ]:
# affichage des types float
num_types = data.select_dtypes(include=['float64','int64','bool']).columns.tolist()
print("Il y a ",len(num_types),"variables numériques")
num_types

#### <font color='skyblue'> Prétraitement des données : approche qualité </font>


Nous corrigons le type des variables :
- DataYear doit être en object car n'est pas une variable quantitative
- ZipCOde idem, le .0 devra être supprimé de ce fait
- CouncilDistrictCode idem

In [ ]:
# changer datayear en object
data['DataYear'] = data['DataYear'].apply(str)

# changer ZipCode en object et supprimer .0
data['ZipCode'] = data['ZipCode'].astype('object').apply(lambda x: str(x).replace('.0',''))
data['ZipCode'] = data['ZipCode'].where(data['ZipCode']!='nan',other=np.nan)

# changer CouncilDistrictCode en object
data['CouncilDistrictCode'] = data['CouncilDistrictCode'].apply(str).astype('object')


Nous corrigons les colonnes dont le taux de remplissage est faible :
- Outlier : remplacer NaN par non-outlier

In [ ]:
# créer valeur Non-outlier, les ouliers seront supprimés par la suite
data['Outlier'] = data['Outlier'].fillna('Non-outlier')
data['Outlier'].value_counts()


In [ ]:
(data['SiteEnergyUseWN(kBtu)']==0).sum()

#### <font color='skyblue'> Prétraitement des données : approche métier </font>


Nous nous intérressons dans ce projet uniquement aux immeubles qui n'ont pas un usage d'habitation.

In [ ]:
data['BuildingType'].unique()

- NonResidential : Cela fait référence à des zones ou des propriétés qui ne sont pas utilisées à des fins résidentielles, mais plutôt pour des activités commerciales, industrielles ou autres.

- Nonresidential COS : Il s'agit probablement d'une sous-catégorie de non-résidentiel qui pourrait spécifier un type spécifique de zonage ou d'utilisation, peut-être quelque chose lié aux services de santé ou aux services professionnels.

- Multifamily MR (5-9) : Cette catégorie indique des zones ou des propriétés réservées à des immeubles d'habitation multifamiliaux de taille moyenne, généralement de 5 à 9 unités.

- SPS-District K-12 : Cela semble être une désignation pour les districts scolaires qui couvrent les écoles primaires et secondaires (de la maternelle à la 12e année).

- Campus : Cela fait référence à des zones ou des propriétés qui abritent des campus universitaires ou d'autres types de complexes éducatifs.

- Multifamily LR (1-4) : Cette catégorie désigne des zones ou des propriétés destinées à des immeubles d'habitation multifamiliaux de petite à moyenne taille, généralement de 1 à 4 unités.

- Multifamily HR (10+) : Il s'agit d'une classification pour des zones ou des propriétés destinées à des immeubles d'habitation multifamiliaux de grande taille, comprenant 10 unités ou plus.

- Nonresidential WA : Cette désignation pourrait indiquer des zones ou des propriétés non résidentielles utilisées à des fins spécifiques telles que l'entreposage ou les installations industrielles, mais la signification exacte de "WA" dépendrait du contexte.

Les variables sélectionnées sont : ['NonResidential', 'Nonresidential COS','SPS-District K-12', 'Campus', 'Nonresidential WA']

In [ ]:
# sélection des catégories d'immeuble
data = data[data['BuildingType'].isin(['NonResidential', 'Nonresidential COS', 
       'SPS-District K-12', 'Campus', 'Nonresidential WA'])]

# affichage de la shape
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))

In [ ]:
# visualisation du count des valeurs :
data['BuildingType'].value_counts()

In [ ]:
# correction du nom d'un individu
data['BuildingType'] = data['BuildingType'].replace('Nonresidential WA','NonResidential')

Nous nous interressons aux valeurs nulles de nos variables :
- SiteEnergyUseWN(kBtu) : supprimer les individus = 0
- TotalGHGEmissions : idem
- NumberofBuildings : idem

In [ ]:
# affichage 0 des colonnes
(data[['SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions','NumberofBuildings']] == 0).sum()

In [ ]:
# suppression des individus = 0 
data = data.dropna(subset=['SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions', 'NumberofBuildings'])
data = data.drop(index=data[(data['SiteEnergyUseWN(kBtu)'] == 0) | (data['TotalGHGEmissions'] == 0) | (data['NumberofBuildings'] == 0)].index)

# affichage 0 des colonnes
(data[['SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions','NumberofBuildings']] == 0).sum()

In [ ]:
# affichage de la shape
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))

Nous supprimons les colonnes inutiles :
- City : toutes les valeurs sont égales à Seattle
- State : toutes les valeurs sont égales à Seattle WA
- DataYear : toutes les valeurs sont égales à 2016
- YearsENERGYSTARCertified : les valeurs ne sont pas exploitables et le taux de remplissage est faible
- Comments : taux de remplissage quasi nul, peu utile
- Suppression de la colonne ENERGYSTARScore car trop de valeurs manquantes
- Suppression de la colonne ZipCode car inutile
- Suppression longitude et latitude

In [ ]:
# suppression des colonnes inutiles
data = data.drop(columns=["City", "State", "DataYear", "YearsENERGYSTARCertified","Comments","ZipCode","Longitude","Latitude"])

# affichage de la shape
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))

Nous traitons maintenant les valeurs aberrantes :
- Outlier : suppression des valeurs Low et High oulier, puis suppression de la colonne
- ComplianceStatus : suppression des valeurs Non-compliant et colonne

In [ ]:
# affichage des valeurs outlier
data['Outlier'].value_counts()

In [ ]:
# affichage des valeurs outlier
data['ComplianceStatus'].value_counts()

In [ ]:
# affichage des valeurs DefaultData
data['DefaultData'].value_counts()

In [ ]:
# suppression des individus ouliers
data = data.loc[(data["Outlier"] == 'Non-outlier')]

# suppression des individus Error 
data = data.loc[data["ComplianceStatus"].isin(['Compliant', 'Error - Correct Default Data'])]

# suppression de la colonne inutile désormais
data = data.drop(columns=["Outlier"])

# suppression de la colonne inutile désormais
data = data.drop(columns=["ComplianceStatus"])

# suppression de la colonne inutile désormais
data = data.drop(columns=["DefaultData"])

Nous affichons les statistiques descriptives des variables numériques afin de repérer les valeurs aberrantes.

In [ ]:
# création liste variables numériques 
numeric_cols  = data.select_dtypes(include=['int', 'float']).columns.tolist()

# Calculer les statistiques descriptives
descriptive_stats = data[numeric_cols].describe()

# Afficher les statistiques descriptives
print(descriptive_stats)

Nous traitons les valeurs aberrantes suivantes :
- NumberofFloors : Selon Wikipédia, le plus grand immeuble de Seattle compte 76 étages, suppression des valeurs sup à 76
- TotalGHGEmissions, GHGEmissionsIntensity, Electricity(kBtu), Electricity(kWh), SourceEUIWN(kBtu/sf) : suppression des valeurs négatives


In [ ]:
# suppression des individus sup à 76
data = data.loc[data['NumberofFloors']<= 76]

In [ ]:
# liste de variable sans longitude pour supprimer les individus négatifs
numeric_cols2 = numeric_cols.copy()

variable_a_supprimer = "Longitude"
if variable_a_supprimer in numeric_cols2:
    numeric_cols2.remove(variable_a_supprimer)
    
    # Filtrer les individus avec au moins une valeur négative dans une colonne numérique
negative_values_numeric = data[(data[numeric_cols2] < 0).any(axis=1)]

# Afficher les individus 
negative_values_numeric

In [ ]:
# Supprimer l'individu aberrant
data = data[data['OSEBuildingID'] != 49784]


# affichage de la shape
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))

Renommage des valeurs de Neighborhood

In [ ]:
# Renommer les valeurs identiques
data['Neighborhood'].replace({
    'Ballard': 'BALLARD',
    'Delridge': 'DELRIDGE',
    'North': 'NORTH',
    'Northwest': 'NORTHWEST',
    'Central': 'CENTRAL',
    'DELRIDGE NEIGHBORHOODS': 'DELRIDGE'
}, inplace=True)

# Afficher les valeurs uniques après la modification
print(data['Neighborhood'].unique())


Nous traitons desormais les valeurs manquantes :
- SecondLargestPropertyUseType : type d'utilisation secondaire de la propriété, mise à "None"
- SecondLargestPropertyUseTypeGFA : superficie d'utilisation secondaire de la propriété, mise à 0
- ThirdLargestPropertyUseType : type d'utilisation tertiaire de la propriété mise à "None"
- ThirdLargestPropertyUseTypeGFA : superficie d'utilisation tertiaire de la propriété, mise à 0
- Supprimer les 4 lignes dont les valeurs sont manquantes
- ENERGYSTARScore : Remplacer les nan par 0


In [ ]:
# Remplacer les valeurs manquantes dans SecondLargestPropertyUseType par "None"
data['SecondLargestPropertyUseType'].fillna("None", inplace=True)

# Remplacer les valeurs manquantes dans SecondLargestPropertyUseTypeGFA par 0
data['SecondLargestPropertyUseTypeGFA'].fillna(0, inplace=True)

# Remplacer les valeurs manquantes dans ThirdLargestPropertyUseType par "None"
data['ThirdLargestPropertyUseType'].fillna("None", inplace=True)

# Remplacer les valeurs manquantes dans ThirdLargestPropertyUseTypeGFA par 0
data['ThirdLargestPropertyUseTypeGFA'].fillna(0, inplace=True)

# Supprimer les lignes avec des valeurs manquantes dans les colonnes spécifiées
data.dropna(subset=['LargestPropertyUseType', 'LargestPropertyUseTypeGFA'], inplace=True)

# Remplacer les nan par 0
data['ENERGYSTARScore'].fillna(0, inplace=True)

# affichage de la shape
print("Ce dataframe contient {} lignes et {} colonnes ".format(data.shape[0], data.shape[1]))


In [ ]:
data_info(data)

#### <font color='skyblue'> Feature Engineering </font>


- Création d'une feature ancienneté du batiment

In [ ]:
#création de la colonne BuildingAge
data['BuildingAge'] = 2016-data['YearBuilt']

- Création d'une feature pourcentage de consommation de gaz et d'éléctricité

In [ ]:
# Créer la nouvelle colonne GasUseRatio(%)
data = data.assign(GasUseRatio = lambda x: (x['NaturalGas(kBtu)'] / x['SiteEnergyUse(kBtu)']))

In [ ]:
# Créer la nouvelle colonne ElectricityUseRatio(%)
data = data.assign(ElectricityUseRatio = lambda x: (x['Electricity(kBtu)'] / x['SiteEnergyUse(kBtu)']))

- Création d'une feature pourcentage de parking

In [ ]:
# Créer la nouvelle colonne GasUseRatio(%)
data = data.assign(ParkingGFARatio = lambda x: (x['PropertyGFAParking'] / x['PropertyGFATotal']))

- Regroupement des variables Building type en 12 catégories

In [ ]:
# Afficher les valeurs uniques de la colonne LargestPropertyUseType
unique_values = data['LargestPropertyUseType'].unique()
print(unique_values)

In [ ]:
def assign_category(property_type):
    if any(word in property_type for word in ['Hospital', 'Clinic', 'Medical', 'Urgent Care', 'Rehabilitation', 'Residential Care', 'Prison']):
        return 'Hospital'
    elif any(word in property_type for word in ['School', 'University', 'Library', 'Dormitory', 'Education', 'Pre-school', 'Daycare', 'Adult']):
        return 'Education'
    elif any(word in property_type for word in ['Hotel', 'Dealership', 'Store', 'Supermarket', 'Restaurant', 'Theater', 'Bank', 'Services', 'Repair']):
        return 'Shopping'
    elif any(word in property_type for word in ['Police', 'Courthouse', 'Office', 'Financial', 'Distribution', 'Parking', 'Fire']):
        return 'Government'
    elif any(word in property_type for word in ['Manufacturing', 'Industrial']):
        return 'Industrial'
    elif any(word in property_type for word in ['Entertainment', 'Museum', 'Performing Arts']):
        return 'Entertainment'
    elif any(word in property_type for word in ['Gym', 'Fitness']):
        return 'Wellness'
    elif any(word in property_type for word in ['Meeting Hall', 'Community', 'Worship']):
        return 'Community'
    elif any(word in property_type for word in ['Data Center', 'Laboratory']):
        return 'Workplace'
    elif any(word in property_type for word in ['Storage', 'Warehouse']):
        return 'Storage'
    elif 'Residential' in property_type:
        return 'Residential'
    elif 'Services' in property_type:
        return 'Services'
    else:
        return 'Miscellaneous'

# Appliquer la fonction assign_category à la colonne "LargestPropertyUseType" pour créer une nouvelle colonne "Category"
data['Category'] = data['LargestPropertyUseType'].apply(assign_category)

# Afficher le DataFrame avec la nouvelle colonne "Category"
data.head(3)



In [ ]:
# Afficher les valeurs uniques de la colonne LargestPropertyUseType
unique_values = data['Category'].unique()

# Afficher les valeurs uniques
print(unique_values)

- Présélection de variables pertinentes

In [ ]:
# Convertir la variable 'OSEBuildingID' en type 'object'
data['OSEBuildingID'] = data['OSEBuildingID'].astype('object')

# Afficher les variables numériques
numeric_variables = data.select_dtypes(include=['int', 'float'])
numeric_variable_names = list(numeric_variables.columns)

numeric_variable_names

- Nous conservons : 
['OSEBuildingID',
 'NumberofBuildings',
 'NumberofFloors',
 'PropertyGFABuilding(s)',
 'LargestPropertyUseTypeGFA',
 'SecondLargestPropertyUseTypeGFA',
 'ThirdLargestPropertyUseTypeGFA',
 'SiteEUI(kBtu/sf)',
 'SourceEUI(kBtu/sf)',
 'SiteEnergyUse(kBtu)',
 'SteamUse(kBtu)',
 'Electricity(kBtu)',
 'NaturalGas(kBtu)',
 'TotalGHGEmissions',
 'GHGEmissionsIntensity',
 'BuildingAge']

In [ ]:
# Afficher les variables numériques
categorical_variables = data.select_dtypes(include=['object'])
categorical_variable_names = list(categorical_variables.columns)
categorical_variable_names

- Nous conservons :
['OSEBuildingID',
 'PrimaryPropertyType',
 'PropertyName',
 'Address',
 'Neighborhood',
 'ListOfAllPropertyUseTypes',
 'LargestPropertyUseType',
 'SecondLargestPropertyUseType',
 'ThirdLargestPropertyUseType']

In [ ]:
# Créer un df avec les variables numériques
categorical_data = data[['Category', 'Neighborhood']]

# Créer un df avec les variables numériques
numeric_data = data[['NumberofFloors','NumberofBuildings','PropertyGFATotal','BuildingAge','GasUseRatio','ElectricityUseRatio','ParkingGFARatio',
                        'ENERGYSTARScore','SiteEnergyUse(kBtu)','TotalGHGEmissions']]


In [ ]:
# Convertir la colonne "NumberofBuildings" en type int64
data['NumberofBuildings'] = data['NumberofBuildings'].astype('int64')

In [ ]:
# Variables à arrondir
variables_to_round = ['GasUseRatio', 'ParkingGFARatio', 'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions','ElectricityUseRatio']

# Arrondir les valeurs à 2 chiffres après la virgule pour les variables spécifiées
data[variables_to_round] = data[variables_to_round].round(2)

#### <font color='skyblue'> Analyse préliminaire </font>


- Observation des distributions 

In [ ]:
# Afficher les distributions
numeric_variables = numeric_data.columns
num_plots_per_row = 4
total_plots = len(numeric_variables)
num_rows = total_plots // num_plots_per_row

if total_plots % num_plots_per_row != 0:
    num_rows += 1

fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(15, 5*num_rows))
plt.subplots_adjust(hspace=0.5)

# créer distribution
for i, variable in enumerate(numeric_variables):
    row_index = i // num_plots_per_row
    col_index = i % num_plots_per_row
    
    if num_rows == 1:
        ax = axes[col_index]
    else:
        ax = axes[row_index, col_index]
    
    sns.histplot(numeric_data[variable], ax=ax, kde=True)
    ax.set_title(variable)
    ax.set_xlabel('')
    ax.set_ylabel('')

plt.show()


In [ ]:
# Créer des boxplots pour chaque variable numérique
plt.figure(figsize=(12, 8))
sns.boxplot(data=numeric_data, orient='h')
plt.title('Boxplots des variables numériques')
plt.xlabel('Valeurs')
plt.show()

- Observation des variables cibles

In [ ]:
# Sélectionner les variables cibles
target_variables = ['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']

# Afficher distribution et boxplot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for i, variable in enumerate(target_variables):
    sns.histplot(data=numeric_data, x=variable, kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f'Distribution de {variable}')
    axes[i, 0].set_xlabel('')
    axes[i, 0].set_ylabel('Fréquence')

for i, variable in enumerate(target_variables):
    sns.boxplot(data=numeric_data, y=variable, ax=axes[i, 1])
    axes[i, 1].set_title(f'Boxplot de {variable}')
    axes[i, 1].set_xlabel('')
    axes[i, 1].set_ylabel('')

plt.tight_layout()

plt.show()


- Tests de normalité

Test Shapiro sur l'ensemble des variables numériques.

In [ ]:
# Créer une liste pour stocker les résultats des tests de normalité
shapiro_tests = []

# Parcourir toutes les colonnes numériques et effectuer le test de Shapiro-Wilk
for column in numeric_data.columns:
    # Effectuer le test de Shapiro-Wilk pour chaque variable
    stat, p_value = shapiro(numeric_data[column])
    
    # Ajouter les résultats du test à la liste
    shapiro_tests.append((column, stat, p_value))

# Afficher les résultats des tests de normalité
print("Résultats des tests de normalité :")
for result in shapiro_tests:
    print(f"Variable : {result[0]}, Statistique de test : {result[1]}, p-value : {result[2]}")


Test Agostino-Pearson sur l'ensemble des variables numériques.

In [ ]:
# Créer une liste pour stocker les résultats des tests de normalité
normaltest_tests = []

# Parcourir toutes les colonnes numériques et effectuer le test de Shapiro-Wilk
for column in numeric_data.columns:
    # Effectuer le test de Shapiro-Wilk pour chaque variable
    stat, p_value = normaltest(numeric_data[column])
    
    # Ajouter les résultats du test à la liste
    normaltest_tests.append((column, stat, p_value))

# Afficher les résultats des tests de normalité
print("Résultats des tests de normalité :")
for result in normaltest_tests:
    print(f"Variable : {result[0]}, Statistique de test : {result[1]}, p-value : {result[2]}")


En se basant sur les résultats, quasiment toutes les variables ont des p-valeurs de 0.0, ce qui signifie que la probabilité d'obtenir ces résultats ou des résultats encore plus extrêmes si les données suivaient une distribution normale est extrêmement faible. Par conséquent, nous rejetterions l'hypothèse nulle pour toutes les variables et concluons que les données ne suivent pas une distribution normale.

In [ ]:
# Calculer la matrice de corrélation
correlation_matrix = numeric_data.corr()

# Créer une heatmap pour visualiser la matrice de corrélation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Matrice de corrélation des variables')
plt.show()


- Normalisation logarithmique

In [ ]:
columns_to_transform = numeric_data.columns

def apply_log(X):
    output = pd.DataFrame(index=X.index)
    for col in X.columns:
        output[col] = np.log(X[col])
    return output

log_transformer = FunctionTransformer(apply_log)

pipeline = ColumnTransformer(transformers=[
    ('log', log_transformer, columns_to_transform)
], remainder='passthrough')  

X = numeric_data
X_transformed = pipeline.fit_transform(X)

X_transformed_df = pd.DataFrame(X_transformed, columns=columns_to_transform.tolist() + X.columns.difference(columns_to_transform).tolist())


In [ ]:
# Affichage des distibutions après transformation
num_plots_per_row = 3
total_plots = len(X_transformed_df.columns)

num_rows = total_plots // num_plots_per_row
if total_plots % num_plots_per_row != 0:
    num_rows += 1

fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(15, 5*num_rows), squeeze=False)
plt.subplots_adjust(hspace=0.5)

for ax in axes.flatten()[total_plots:]:
    ax.axis('off')

for i, variable in enumerate(X_transformed_df.columns):
    row_index = i // num_plots_per_row
    col_index = i % num_plots_per_row
    ax = axes[row_index, col_index]
    
    sns.histplot(X_transformed_df[variable], ax=ax, kde=True, stat="density")
    ax.set_title(variable)
    ax.set_xlabel('')
    ax.set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Sélectionner les variables cibles
target_variables = ['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']

# Appliquer une transformation logarithmique aux variables cibles
for variable in target_variables:
    numeric_data[variable] = np.log(numeric_data[variable])

# Afficher distribution et boxplot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for i, variable in enumerate(target_variables):
    sns.histplot(data=numeric_data, x=variable, kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f'Distribution de {variable}')
    axes[i, 0].set_xlabel('')
    axes[i, 0].set_ylabel('Fréquence')

for i, variable in enumerate(target_variables):
    sns.boxplot(data=numeric_data, y=variable, ax=axes[i, 1])
    axes[i, 1].set_title(f'Boxplot de {variable}')
    axes[i, 1].set_xlabel('')
    axes[i, 1].set_ylabel('')

plt.tight_layout()

plt.show()


In [ ]:
# liste pour stocker les résultats des tests de normalité
shapiro_tests = []

for column in X_transformed_df.columns:
    stat, p_value = shapiro(X_transformed_df[column])
    shapiro_tests.append((column, stat, p_value))

print("Résultats des tests de normalité :")
for result in shapiro_tests:
    print(f"Variable : {result[0]}, Statistique de test : {result[1]}, p-value : {result[2]}")

In [ ]:
data.to_csv('./data.csv', sep=';', encoding='utf-8')

#### <font color='skyblue'> Analyse des variables catégorielles </font>


In [ ]:
categorical_data = ['Category', 'Neighborhood']
numerical_data = ['NumberofFloors','PropertyGFATotal','NumberofBuildings','BuildingAge','GasUseRatio','ParkingGFARatio',
                'ENERGYSTARScore','SiteEnergyUse(kBtu)','TotalGHGEmissions']

- Neighborhood

In [ ]:
# Visualisation de Neighborhood
ax, fig = plt.subplots(figsize=(12,8)) 
ax = sns.countplot(data=data, y='Neighborhood', alpha=.8,
              order=data['Neighborhood'].value_counts().index, orient='horizontal')
ax.set_title("Batîment Neighborhood de Seattle")
plt.show()

In [ ]:
# visualisation de TotalGHGEmissions en fonction de Neighborhood
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

sns.boxplot(x='Neighborhood', y='TotalGHGEmissions', data=data, ax=axes[0])
axes[0].set_title('Distribution des émissions totales de gaz à effet de serre par quartier')
axes[0].set_xlabel('Quartier')
axes[0].set_ylabel('Émissions totales de gaz à effet de serre')
axes[0].tick_params(axis='x', rotation=45) 

sns.barplot(x='Neighborhood', y='TotalGHGEmissions', data=data, ax=axes[1], ci=None)
axes[1].set_title('Moyenne des émissions totales de gaz à effet de serre par quartier')
axes[1].set_xlabel('Quartier')
axes[1].set_ylabel('Consommation d\'énergie du site (kBtu)')
axes[1].tick_params(axis='x', rotation=45)  

plt.tight_layout()
plt.show()


In [ ]:
# visualisation de SiteEnergyUse(kBtu) en fonction de Neighborhood
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

sns.boxplot(x='Neighborhood', y='SiteEnergyUse(kBtu)', data=data, ax=axes[0])
axes[0].set_title('Distribution de la consommation en énergie en fonction des quartiers')
axes[0].set_xlabel('Quartier')
axes[0].set_ylabel('Émissions totales de gaz à effet de serre')
axes[0].tick_params(axis='x', rotation=45) 

sns.barplot(x='Neighborhood', y='SiteEnergyUse(kBtu)', data=data, ax=axes[1], ci=None)
axes[1].set_title('Moyenne de la consommation en énergie en fonction des quartiers ')
axes[1].set_xlabel('Quartier')
axes[1].set_ylabel('Consommation d\'énergie du site (kBtu)')
axes[1].tick_params(axis='x', rotation=45)  

plt.tight_layout()
plt.show()


- LargestPropertyUseType

In [ ]:
# Visualisation de LargestPropertyUseType
ax, fig = plt.subplots(figsize=(12,8)) 
ax = sns.countplot(data=data, y='Category', alpha=.9,
              order=data['Category'].value_counts().index, orient='horizontal')
ax.set_title("Usage des batiment de Seattle")
plt.show()

In [ ]:
# visualisation de TotalGHGEmissions en fonction de Category
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

sns.boxplot(x='Category', y='TotalGHGEmissions', data=data, ax=axes[0])
axes[0].set_title('Distribution des émissions totales de gaz à effet de serre en fonction de l\'usage')
axes[0].set_xlabel('Quartier')
axes[0].set_ylabel('Émissions totales de gaz à effet de serre')
axes[0].tick_params(axis='x', rotation=45) 

sns.barplot(x='Category', y='TotalGHGEmissions', data=data, ax=axes[1], ci=None)
axes[1].set_title('Moyenne des émissions totales de gaz à effet de serre en fonction de l\'usage')
axes[1].set_xlabel('Quartier')
axes[1].set_ylabel('Consommation d\'énergie du site (kBtu)')
axes[1].tick_params(axis='x', rotation=45)  

plt.tight_layout()
plt.show()

In [ ]:
# visualisation de SiteEnergyUse(kBtu) en fonction de Category
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

sns.boxplot(x='Category', y='SiteEnergyUse(kBtu)', data=data, ax=axes[0])
axes[0].set_title('Distribution de la consommation en énergie en fonction de l\'usage')
axes[0].set_xlabel('Quartier')
axes[0].set_ylabel('Émissions totales de gaz à effet de serre')
axes[0].tick_params(axis='x', rotation=45) 

sns.barplot(x='Category', y='SiteEnergyUse(kBtu)', data=data, ax=axes[1], ci=None)
axes[1].set_title('Moyenne de la consommation en énergie en fonction de l\'usage')
axes[1].set_xlabel('Quartier')
axes[1].set_ylabel('Consommation d\'énergie du site (kBtu)')
axes[1].tick_params(axis='x', rotation=45)  

plt.tight_layout()
plt.show()

- Catégories de batiment par quartier

In [ ]:
# Créer une figure et un axe pour le graphique
plt.figure(figsize=(10, 6))

# Créer un graphique en barres pour visualiser les valeurs des variables qualitatives
sns.countplot(x='Neighborhood', hue='Category', data=data)

# Ajouter un titre et des labels aux axes
plt.title('Répartition des catégories par quartier')
plt.xlabel('Quartier')
plt.ylabel('Nombre de bâtiments')

# Faire pivoter les labels sur l'axe x pour une meilleure lisibilité
plt.xticks(rotation=45, ha='right')

# Afficher la légende
plt.legend(title='Catégorie')

# Afficher le graphique
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Créer une figure et un axe pour le graphique
plt.figure(figsize=(20, 10))

# Grouper les données par quartier et catégorie, puis compter le nombre de bâtiments dans chaque groupe
grouped_data = data.groupby(['Neighborhood', 'Category']).size().unstack(fill_value=0)

# Limiter le nombre de quartiers à afficher pour correspondre au nombre de sous-graphiques
grouped_data = grouped_data.head(6)  

# Créer un diagramme à secteurs pour chaque quartier
for neighborhood in grouped_data.index:
    plt.subplot(3, 2, grouped_data.index.tolist().index(neighborhood) + 1)
    plt.pie(grouped_data.loc[neighborhood], labels=grouped_data.columns, autopct='%1.1f%%', startangle=140)
    plt.title(neighborhood)

# Ajuster la disposition pour éviter les chevauchements
plt.tight_layout()

# Afficher le graphique
plt.show()
